## 1. Import Necessary Libraries

In [10]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import metrics
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import ConvNeXtBase
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

## 2. Load Data + Data Preprocessing

In [3]:
# Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')
print(train_df.shape[0])

# Generate file paths correctly
train_df['file_path'] = train_df.apply(
    lambda row: f"/kaggle/input/bttai-ajl-2025/train/train/{row['label']}/{row['md5hash']}.jpg", axis=1
)
test_df['file_path'] = test_df['md5hash'].apply(
    lambda x: f"/kaggle/input/bttai-ajl-2025/test/test/{x}.jpg"
)

# Data Preprocessing

# Remove invalid rows
train_df = train_df[(train_df['fitzpatrick_scale'] > 0) & (train_df['label'].notna())]
print(train_df.shape[0])

train_df = train_df[train_df['file_path'].apply(os.path.exists)]
print(train_df.shape[0])

test_df = test_df[test_df['file_path'].apply(os.path.exists)]
print(test_df.shape[0])
print()


# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])


# Splitting dataset into training and validation datasets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['encoded_label'])

# Define image data generators for training and testing
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.9, 1.1],
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=512,
    class_mode='raw',
    shuffle = True
)


val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=512,
    class_mode='raw',
    shuffle=False
    
)


test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    target_size=(224, 224),
    batch_size= 512,
    class_mode=None,
    shuffle=False
    
)


2860
2752
2752
1227

Found 2201 validated image filenames.
Found 551 validated image filenames.
Found 1227 validated image filenames.


In [4]:
# Common Model Parameters
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df['encoded_label']),
    y=train_df['encoded_label']
)
class_weights_dict = dict(enumerate(class_weights))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Initialize the ReduceLROnPlateau callback
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 factor=0.5,  # Factor to reduce the learning rate
                                 patience=3,  # Number of epochs to wait before reducing
                                 min_lr=1e-6)  # Minimum learning rate

## 4. Model Training

In [17]:
# Load ConvNeXtTiny with pre-trained weights
base_model = ConvNeXtBase(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model


# Build the model
model = models.Sequential([
    base_model,  # Base model (ConvNeXtTiny)
    layers.GlobalAveragePooling2D(),  # Pooling layer to reduce spatial dimensions
    layers.BatchNormalization(),  # Batch normalization to stabilize training
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # First dense layer with more units
    layers.Dropout(0.4),  # Increased dropout rate to prevent overfitting
    layers.BatchNormalization(),  # Another batch normalization layer
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # Second dense layer
    layers.Dropout(0.4),  # Dropout layer for regularization
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),  # Third dense layer
    layers.Dropout(0.3),  # Dropout layer
    layers.Dense(21, activation='softmax')  # Output layer with 21 classes (adjust accordingly)
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[lr_reduction])

350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 117s 14s/step - accuracy: 0.0621 - loss: 4.2286 - val_accuracy: 0.2033 - val_loss: 3.4399 - learning_rate: 0.0010
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.2156 - loss: 3.3747 - val_accuracy: 0.2722 - val_loss: 3.2084 - learning_rate: 0.0010
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.2841 - loss: 3.1589 - val_accuracy: 0.3013 - val_loss: 3.0965 - learning_rate: 0.0010
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.3206 - loss: 2.9238 - val_accuracy: 0.3394 - val_loss: 3.0449 - learning_rate: 0.0010
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.3382 - loss: 2.8204 - val_accuracy: 0.3575 - val_loss: 2.9785 - learning_rate: 0.0010
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.3766 - loss: 2.6938 - val_accuracy: 0.3557 - val_loss: 2.9034 - learning_rate: 0.0010
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accurac

In [18]:
# Generate predictions
y_prob = model.predict(val_generator)
y_pred = np.argmax(y_prob, axis=1)
y_true = val_df['encoded_label'].values

# Print classification report
print(classification_report(y_true, y_pred))

f1 = f1_score(y_true, y_pred, average='weighted')
print("ConvNeXtTiny F1 Score:", f1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 5s/step
              precision    recall  f1-score   support

           0       0.28      0.19      0.23        26
           1       0.60      0.74      0.67        47
           2       0.28      0.21      0.24        24
           3       0.48      0.75      0.58        64
           4       0.38      0.38      0.38         8
           5       0.42      0.45      0.43        11
           6       0.65      0.52      0.58        21
           7       0.57      0.36      0.44        11
           8       0.42      0.46      0.44        28
           9       0.86      0.50      0.63        12
          10       0.65      0.73      0.69        44
          11       0.75      0.57      0.65        21
          12       0.62      0.48      0.54        21
          13       0.14      0.07      0.09        15
          14       0.59      0.50      0.54        34
          15       0.57      0.64      0.60        25
          16       0.69      0.75      0.72 

In [19]:
# SUBMISSION.CSV
y_pred = np.argmax(model.predict(test_generator), axis = 1)
test_df['label'] = label_encoder.inverse_transform(y_pred)

# Save submission
test_df[['md5hash', 'label']].to_csv('/kaggle/working/submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 8s/step
